In [1]:
import numpy as np
from scipy.stats import gamma
from scipy.stats import multivariate_normal
from scipy.stats import norm
import matplotlib.pyplot as plt
import math

In [19]:
'''
STEP 1:

We define preliminary measurements and associate points on the dartboard with regions on the dartboard.

'''


In [2]:
# Define the dartboard parameters
bullseye_inner_radius = 12.7
bullseye_outer_radius = 31.8
triple_ring_inner_radius = 99
triple_ring_outer_radius = 107
double_ring_inner_radius = 162
double_ring_outer_radius = 170

In [3]:
#DICTIONARY ASSOCIATING THE CIRCULAR ORDERING (1-20) beginning at 13 TO THE POINTS OF THE S REGION 
trans_angle_to_score_region = {
        1: 13,
        2: 4,
        3: 18,
        4: 1,
        5: 20,
        6: 5,
        7: 12,
        8: 9,
        9: 14,
        10: 11,
        11: 8,
        12: 16,
        13: 7,
        14: 19,
        15: 3,
        16: 17,
        17: 2,
        18: 15,
        19: 10,
        20: 6
    }
score_region_to_trans_angle = {value: key for key, value in trans_angle_to_score_region.items()}

In [4]:
def check_score_region(x, y):
    distance = np.sqrt(x**2 + y**2)
    angle = np.arctan2(y, x) % (2 * np.pi)
    trans_angle = int((((angle - (np.pi / 20))/(np.pi / 10))) % 20) + 1 #converts angle into 1-20 for usage
    if distance <= bullseye_inner_radius:
        return 'BULL'
    elif distance <= bullseye_outer_radius:
        return 'BULLO'
    elif distance <= triple_ring_inner_radius:
        return f'S{trans_angle_to_score_region[trans_angle]}'
    elif distance <= triple_ring_outer_radius:
        return f'T{trans_angle_to_score_region[trans_angle]}'
    elif distance <= double_ring_inner_radius:
        return f'S{trans_angle_to_score_region[trans_angle]}'
    elif distance <= double_ring_outer_radius:
        return f'D{trans_angle_to_score_region[trans_angle]}'
    else:
        return 'OOB'

In [5]:
print(check_score_region(100, 100))

S18


In [6]:
#given a region, returns the number of points - will be used in generating heatmap
def region_to_points(region):
    if region == 'BULL': return 50
    if region == 'BULLO': return 25
    if region == 'OOB': return 0
    
    multiplier_dict = {
        'S': 1,
        'D': 2,
        'T': 3
    }
    mult_value = multiplier_dict[region[0]]
    raw_value = int(region[1:])
    return mult_value * raw_value

In [7]:
print(region_to_points(check_score_region(50,86)))

54


In [8]:
# GENERATE ALL REGIONS OF THE BOARD
combinations = [f"{letter}{number}" for letter in ['S', 'D', 'T'] for number in range(1, 21)]
additional_entries = ['OOB', 'BULL', 'BULLO']
dartboard = combinations + additional_entries

# Print the resulting list
print(dartboard)

['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'OOB', 'BULL', 'BULLO']


In [9]:
def region_to_center(region):
    '''
    ONLY WORKS FOR BULL, BULLO, D, T regions
    FOR EVERYTHING ELSE, it returns (0,0) which is not good practice, but whatever
    '''
    if region == 'BULL': return (0,0)
    if region == 'BULLO': return (0,0)
    #ANGLE
    raw_value = int(region[1:])
    trans_angle = score_region_to_trans_angle[raw_value]
    angle = ((trans_angle)%20)*(np.pi/10)
    
    #DISTANCE   
    if region[0] == 'D': distance = (double_ring_inner_radius + double_ring_outer_radius)/2
    if region[0] == 'T': distance = (triple_ring_inner_radius + triple_ring_outer_radius)/2
        
    return (distance*np.cos(angle), distance*np.sign(angle))

In [10]:
region_to_center("D16")

(-134.29682106624128, 166.0)

In [12]:
'''
STEP 2

Now, we generate the probabilities of landing in each of the regions. We take this in steps.

1) Compute the probabilities of landing in a square.
2) Compute the probabilities of landing in some region.
'''

'\nSTEP 2\n\nNow, we generate the probabilities of landing in each of the regions. We take this in steps.\n\n1) Compute the probabilities of landing in a square.\n2) Compute the probabilities of landing in some region.\n'

In [13]:
def probability_in_rectangular_region(mu_x, mu_y, sigma, region):
    # Define the multivariate normal distribution
    mvn_distribution = multivariate_normal([mu_x, mu_y], sigma**2 * np.eye(2))

    # Extract the region bounds
    x_min, x_max, y_min, y_max = region

    # Compute the probability of landing in the rectangular region
    probability =    mvn_distribution.cdf([x_max, y_max]) - mvn_distribution.cdf([x_min, y_max]) - \
                     mvn_distribution.cdf([x_max, y_min]) + mvn_distribution.cdf([x_min, y_min])
    
    return probability

In [14]:
probability_in_rectangular_region(2,3,40,(1,2,3,4))

9.945111915435345e-05

In [15]:
def generate_grid(num_partitions):
    # Preallocate the grid array
    grid = np.empty((num_partitions**2, 4))

    # Compute the size of each partition
    partition_size = 340 / num_partitions

    # Loop through each grid cell and compute the probability of landing in it
    for i in range(num_partitions):
        for j in range(num_partitions):
            # Compute the bounds of the current grid cell
            x_min = -170 + i * partition_size  # Adjusting for the center of the square
            x_max = x_min + partition_size
            y_min = -170 + j * partition_size  # Adjusting for the center of the square
            y_max = y_min + partition_size
            
            # Assign the bounds to the grid array
            index = i * num_partitions + j
            grid[index] = (x_min, x_max, y_min, y_max)
    return grid

In [16]:
generate_grid(4)

array([[-170.,  -85., -170.,  -85.],
       [-170.,  -85.,  -85.,    0.],
       [-170.,  -85.,    0.,   85.],
       [-170.,  -85.,   85.,  170.],
       [ -85.,    0., -170.,  -85.],
       [ -85.,    0.,  -85.,    0.],
       [ -85.,    0.,    0.,   85.],
       [ -85.,    0.,   85.,  170.],
       [   0.,   85., -170.,  -85.],
       [   0.,   85.,  -85.,    0.],
       [   0.,   85.,    0.,   85.],
       [   0.,   85.,   85.,  170.],
       [  85.,  170., -170.,  -85.],
       [  85.,  170.,  -85.,    0.],
       [  85.,  170.,    0.,   85.],
       [  85.,  170.,   85.,  170.]])

In [21]:
import numpy as np
from scipy.stats import multivariate_normal

def compute_total_probability(mu, sigma):
    # Define the multivariate normal distribution
    mvn_distribution = multivariate_normal(mean=mu, cov=sigma**2 * np.eye(2))
    
    # Generate grid
    grid = generate_grid(50)
    
    # Compute centers of grid cells
    x_centers = (grid[:, 0] + grid[:, 1]) / 2
    y_centers = (grid[:, 2] + grid[:, 3]) / 2
    
    # Compute probabilities for all grid cells
    probabilities = mvn_distribution.cdf(np.column_stack((grid[:, 1], grid[:, 3]))) - \
                    mvn_distribution.cdf(np.column_stack((grid[:, 0], grid[:, 3]))) - \
                    mvn_distribution.cdf(np.column_stack((grid[:, 1], grid[:, 2]))) + \
                    mvn_distribution.cdf(np.column_stack((grid[:, 0], grid[:, 2])))
    
    # Compute region numbers for all grid cells
    region_numbers = np.array([check_score_region(x_center, y_center) for x_center, y_center in zip(x_centers, y_centers)])
    
    # Initialize dictionary to store probabilities for each region
    probability_dictionary_int = {i: 0 for i in dartboard}
    
    # Accumulate probabilities for each region
    for region_number in np.unique(region_numbers):
        idx = region_numbers == region_number
        probability_dictionary_int[region_number] = np.sum(probabilities[idx])
    
    return probability_dictionary_int

In [22]:
# Define mean and standard deviation for the multivariate normal distribution
mu = np.array([100, 100])  # Mean
sigma = 1  # Standard deviation

test = (compute_total_probability(mu, sigma))
res = 0
for entry in test:
    res += test[entry]
print(test)
print(res)

    

{'S1': 0.0, 'S2': 0.0, 'S3': 0.0, 'S4': 0.022233338723722115, 'S5': 0.0, 'S6': 0.0, 'S7': 0.0, 'S8': 0.0, 'S9': 0.0, 'S10': 0.0, 'S11': 0.0, 'S12': 0.0, 'S13': 0.0, 'S14': 0.0, 'S15': 0.0, 'S16': 0.0, 'S17': 0.0, 'S18': 0.9777666612762764, 'S19': 0.0, 'S20': 0.0, 'D1': 1.0904161207021266e-224, 'D2': 0.0, 'D3': 0.0, 'D4': 0.0, 'D5': 0.0, 'D6': 0.0, 'D7': 0.0, 'D8': 0.0, 'D9': 0.0, 'D10': 0.0, 'D11': 0.0, 'D12': 0.0, 'D13': 0.0, 'D14': 0.0, 'D15': 0.0, 'D16': 0.0, 'D17': 0.0, 'D18': 4.47551033505493e-17, 'D19': 0.0, 'D20': 0.0, 'T1': 0.0, 'T2': 0.0, 'T3': 0.0, 'T4': 1.3155075082039464e-215, 'T5': 0.0, 'T6': 0.0, 'T7': 0.0, 'T8': 0.0, 'T9': 0.0, 'T10': 0.0, 'T11': 0.0, 'T12': 0.0, 'T13': 0.0, 'T14': 0.0, 'T15': 0.0, 'T16': 0.0, 'T17': 0.0, 'T18': 1.3155075082039464e-215, 'T19': 0.0, 'T20': 0.0, 'OOB': 2.2377551675274626e-16, 'BULL': 0.0, 'BULLO': 0.0}
0.9999999999999988
